## Stage 3. 데이터 전처리 (1)

[목표]

날짜 데이터 전처리

날짜와 Join할 수 있는 데이터 추가

### 1. 데이터 읽어오기

In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None

train_path = '../data/KOREAN_train4.csv'
test_path  = '../data/KOREAN_test4.csv'
subway_feature_path = '../data/subway_feature.csv'
bus_feature_path = '../data/bus_feature.csv'
interest_rate_path = '../data/한국은행_금리.csv'
GDP_path = '../data/국내총생산 및 경제성장률(GDP)_한국은행.csv'
sample_submission_path = '../data/sample_submission.csv'

dt_train = pd.read_csv(train_path)
dt_test = pd.read_csv(test_path)
dt_subway_feature = pd.read_csv(subway_feature_path)
dt_bus_feature = pd.read_csv(bus_feature_path)
dt_interest_rate = pd.read_csv(interest_rate_path)
dt_GDP = pd.read_csv(GDP_path)
dt_sample_submission = pd.read_csv(sample_submission_path)

dt_train['train_test'] = 'train'
dt_test['train_test'] = 'test'
all_data = pd.concat([dt_train, dt_test])
all_data = all_data.reset_index(drop=True)

dt_train.head()

/tmp/ipykernel_469482/639869550.py:12: DtypeWarning: Columns (10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  dt_train = pd.read_csv(train_path)


,시군구,번지,아파트명,전용면적,계약년월,계약일,층,건축년도,도로명,해제사유발생일,...,전세가율,전체계약일자,금리,실질gdp,명목gdp,학군,재개발개수,학급수,여당득표율,train_test
0,서울특별시 강남구 개포동,658-1,개포6차우성,79.97,201712,8,3,1987,언주로 3,NaN,...,55.88,20171208,1.5,2.9,"474,957.90",8,0,29.393939,35.36,train
1,서울특별시 강남구 개포동,658-1,개포6차우성,79.97,201712,22,4,1987,언주로 3,NaN,...,55.88,20171222,1.5,2.9,"474,957.90",8,0,29.393939,35.36,train
2,서울특별시 강남구 개포동,658-1,개포6차우성,54.98,201712,28,5,1987,언주로 3,NaN,...,55.88,20171228,1.5,2.9,"474,957.90",8,0,29.393939,35.36,train
3,서울특별시 강남구 개포동,658-1,개포6차우성,79.97,201801,3,4,1987,언주로 3,NaN,...,54.42,20180103,1.5,2.9,"449,357.80",8,0,30.781250,35.36,train
4,서울특별시 강남구 개포동,658-1,개포6차우성,79.97,201801,8,2,1987,언주로 3,NaN,...,54.42,20180108,1.5,2.9,"449,357.80",8,0,30.781250,35.36,train


In [2]:
dt_train.columns

Index(['시군구', '번지', '아파트명', '전용면적', '계약년월', '계약일', '층', '건축년도', '도로명',
       '해제사유발생일', '거래유형', '중개사소재지', 'k-단지분류', 'k-세대타입(분양형태)', 'k-복도유형',
       'k-난방방식', 'k-전체동수', 'k-전체세대수', 'k-건설사', 'k-시행사', 'k-연면적', 'k-주거전용면적',
       'k-전용면적별세대현황60이하', 'k-전용면적별세대현황6085이하', 'k-85135이하', 'k-135초과',
       '세대전기계약방법', '건축면적', '주차대수', '기타의무임대1234', '단지신청일', 'target', '구', '동',
       '전체주소명', '동아파트명', '유동인구', '아파트 지하철역 거리', '1차역세권', '2차역세권',
       '아파트 버스정류장 거리', '좌표X', '좌표Y', '아파트 한강대교 거리', '전세가율', '전체계약일자', '금리',
       '실질gdp', '명목gdp', '학군', '재개발개수', '학급수', '여당득표율', 'train_test'],
      dtype='object')

### 2. 날짜 데이터

In [3]:
def preprocess_tran_date(x):
    if type(x) == int:
        if x < 10:
            return '0'+str(x)
        else:
            return str(x)
    else:
        return x

all_data['계약일'] = all_data['계약일'].apply(preprocess_tran_date)
all_data['계약날짜'] = all_data['계약년월'].astype(int).astype(str) + all_data['계약일'].astype(str)
all_data['계약날짜'] = pd.to_datetime(all_data['계약날짜'])
all_data = all_data.sort_values('계약날짜').reset_index(drop=True)

### 3. 아파트 키값 생성

In [4]:
all_data['apartment_id'] = all_data.groupby(['시군구', '아파트명']).ngroup()
all_data['apartment_id']

0           747.0
1          3215.0
2          4414.0
3          5768.0
4          3980.0
            ...  
1128089    4475.0
1128090    3318.0
1128091    6782.0
1128092    5137.0
1128093    5962.0
Name: apartment_id, Length: 1128094, dtype: float64

- 아파트 고유 키값을 만들어 문자열 형태가 아닌 숫자를 통해 아파트를 조회할 수 있도록 함.

### 4. 아파트 실거래가 데이터 전처리

In [5]:
def str_to_int(string):
    if type(string) == str:
        string = string.replace(',', '')
        return int(string)
    else:
        return string

all_data['target'] = all_data['target'].apply(str_to_int)

### 5. 최근 아파트 가격 데이터

In [6]:
all_data.columns

Index(['시군구', '번지', '아파트명', '전용면적', '계약년월', '계약일', '층', '건축년도', '도로명',
       '해제사유발생일', '거래유형', '중개사소재지', 'k-단지분류', 'k-세대타입(분양형태)', 'k-복도유형',
       'k-난방방식', 'k-전체동수', 'k-전체세대수', 'k-건설사', 'k-시행사', 'k-연면적', 'k-주거전용면적',
       'k-전용면적별세대현황60이하', 'k-전용면적별세대현황6085이하', 'k-85135이하', 'k-135초과',
       '세대전기계약방법', '건축면적', '주차대수', '기타의무임대1234', '단지신청일', 'target', '구', '동',
       '전체주소명', '동아파트명', '유동인구', '아파트 지하철역 거리', '1차역세권', '2차역세권',
       '아파트 버스정류장 거리', '좌표X', '좌표Y', '아파트 한강대교 거리', '전세가율', '전체계약일자', '금리',
       '실질gdp', '명목gdp', '학군', '재개발개수', '학급수', '여당득표율', 'train_test', '계약날짜',
       'apartment_id'],
      dtype='object')

In [7]:
from datetime import datetime
from tqdm import tqdm

def make_area_bucket(area):
    if area < 60: # 59타입
        return 0
    elif area < 85: # 84타입
        return 1
    else:
        return 2

all_data['bucket_area'] = all_data['전용면적'].apply(make_area_bucket)

def get_recent_price(idx, all_data):
    row = all_data.loc[idx]
    temp_df = all_data.loc[:idx]
    temp_df = temp_df[
        (temp_df['계약날짜'] < row['계약날짜']) &
        (temp_df['bucket_area'] == row['bucket_area'])
    ]
    if len(temp_df) == 0:
        temp_df = all_data[
            (all_data['계약날짜'] < datetime.strptime('2016-01-01', "%Y-%m-%d")) &
            (all_data['bucket_area'] == row['bucket_area'])
        ]
    
    # 아파트 아이디 같은것 찾기
    recent_price = temp_df[(temp_df['apartment_id'] == row['apartment_id'])]
    if len(recent_price) == 0:
        recent_price = temp_df[(temp_df['시군구'] == row['시군구'])]
        if len(recent_price) > 0:
            recent_price = recent_price.iloc[-1]['target']
        else:
            recent_price = None
    else:
        recent_price = recent_price.iloc[-1]['target']  
    
    if recent_price is None:
        recent_price = temp_df['target'].mean() # 2019년 전체평균

    return recent_price

for idx, row in tqdm(all_data.iterrows(), total = all_data.shape[0]):
    if row['train_test'] == 'test':  
        continue
    all_data.loc[idx, 'recent_price'] = get_recent_price(idx, all_data)

100%|██████████| 1128094/1128094 [17:54:26<00:00, 17.50it/s]   


In [8]:
# 결과 데이터프레임을 CSV 파일로 저장
output_filename = "../data/recent_price.csv"
all_data.to_csv(output_filename, index=False)

print(f"Data saved to {output_filename}")

Data saved to ../data/recent_price.csv


- 부동산에서 보통 아파트 시장 가격을 평가할 때, 최근 거래된 아파트 가격을 기준으로 평가하는 경우가 많음.
- 최근 거래를 바탕으로 계산된 아파트 최근 거래가를 이용하여 feature 를 새로 생성하도록 함.

### 6. 아파트 최근 거래량

In [9]:
from datetime import datetime, timedelta
from tqdm import tqdm

for idx, row in tqdm(all_data.iterrows(), total = all_data.shape[0]):
    # 결측값 근처시점 데이터로 처리
    if row['계약날짜'] <= datetime.strptime('2014-03-30', "%Y-%m-%d"):
        start_day = datetime.strptime('2014-03-30', "%Y-%m-%d")
        end_day = datetime.strptime('2014-01-01', "%Y-%m-%d")
        cnt = len(all_data[(all_data['계약날짜'] >= start_day) & (all_data['계약날짜'] < end_day) & (all_data['시군구'] == row['시군구'])])
    else:
        start_day = row['계약날짜'] - timedelta(days=90)
        end_day = row['계약날짜']
        cnt = len(all_data[(all_data['계약날짜'] >= start_day) & (all_data['계약날짜'] < end_day) & (all_data['시군구'] == row['시군구'])])
    
    all_data.loc[idx, 'transaction_cnt'] = cnt

100%|██████████| 1128094/1128094 [18:43:28<00:00, 16.74it/s]  


In [10]:
# 결과 데이터프레임을 CSV 파일로 저장
output_filename = "../data/transaction_cnt.csv"
all_data.to_csv(output_filename, index=False)

print(f"Data saved to {output_filename}")

Data saved to ../data/transaction_cnt.csv


- 거래량이 증가할 수록 아파트의 가격이 증가할 것이라 예상함
- 최근 3개월 동안의 데이터를 활용하여 같은 동네의 아파트 거래량을 카운트 해봄.

### 7. 금리 데이터 불러오기 및 전처리

- 한국은행 금리 데이털르 활용하여 아파트 가격과 관련이 높은 금리 데이터를 추가함

In [ ]:
interest_rate = pd.read_csv(interest_rate_path) # 한국은행 기준금리

def make_date(row):
    month_day = row['월일'].replace('월 ', '-')
    month_day = month_day.replace('일', '')
    date = str(row['연도'])+ '-' + month_day
    return date

interest_rate['날짜'] = interest_rate.apply(lambda x: make_date(x), axis=1)
interest_rate['날짜'] = pd.to_datetime(interest_rate['날짜'])

### 12개월 전 금리

- 금리 데이터를 불러와 1년 전 금리를 적용시켜 데이터를 결합시킴

In [18]:
import openpyxl

goldwe = pd.read_excel('../data/한국은행_금리 (1).xlsx')
goldwe["계약날짜"] = goldwe["변경일자_년"].astype(str) + goldwe["변경일자_월일"].map(lambda x : x[:2] + x[4:6])
g01 = goldwe.sort_values('계약날짜')
g01['날짜열_1년후'] = pd.to_datetime(g01['계약날짜']) + pd.DateOffset(years=1)
g01['날짜열_11개월후'] = pd.to_datetime(g01['계약날짜']) + pd.DateOffset(months=11)
g01['날짜열_13개월후'] = pd.to_datetime(g01['계약날짜']) + pd.DateOffset(months=13)
g02 = g01[['날짜열_11개월후', '기준금리']]
g03 = g01[['날짜열_1년후', '기준금리']]
g04 = g01[['날짜열_13개월후', '기준금리']]

g02.columns = ['날짜열_11개월후', "11개월전_금리"]
g03.columns = ['날짜열_1년후', "1년전_금리"]
g04.columns = ['날짜열_13개월후', "13개월전_금리"]

g02['날짜열_11개월후_종료일'] = g02['날짜열_11개월후'].shift(-1, fill_value = "2027-01-01") - pd.DateOffset(days=1)
g03['날짜열_1년후_종료일'] = g03['날짜열_1년후'].shift(-1, fill_value = "2027-01-01") - pd.DateOffset(days=1)
g04['날짜열_13개월후_종료일'] = g04['날짜열_13개월후'].shift(-1, fill_value = "2027-01-01") - pd.DateOffset(days=1)
make_date = all_data["계약날짜"].reset_index()
make_date_test = all_data["계약날짜"].reset_index()

make_date['계약날짜'] = make_date['계약날짜'].astype('str')
make_date_test['계약날짜'] = make_date_test['계약날짜'].astype('str')

m02 = pd.merge(make_date, g02, how = 'cross')
m03 = pd.merge(make_date, g03, how = 'cross')
m03_test = pd.merge(make_date_test, g03, how = 'cross')
m04 = pd.merge(make_date, g04, how = 'cross')
m02['계약날짜'] = pd.to_datetime(m02['계약날짜'])
m03['계약날짜'] = pd.to_datetime(m03['계약날짜'])
m03_test['계약날짜'] = pd.to_datetime(m03_test['계약날짜'])
m04['계약날짜'] = pd.to_datetime(m04['계약날짜'])
m02['현재와시작일차이'] = (m02['계약날짜'] - m02['날짜열_11개월후']).dt.days
m02['현재와종료일차이'] = (m02['날짜열_11개월후_종료일'] - m02['계약날짜']).dt.days
m02['체크'] = m02['현재와시작일차이']*m02['현재와종료일차이']

m03['현재와시작일차이'] = (m03['계약날짜'] - m03['날짜열_1년후']).dt.days
m03['현재와종료일차이'] = (m03['날짜열_1년후_종료일'] - m03['계약날짜']).dt.days
m03['체크'] = m03['현재와시작일차이']*m03['현재와종료일차이']

m03_test['현재와시작일차이'] = (m03_test['계약날짜'] - m03_test['날짜열_1년후']).dt.days
m03_test['현재와종료일차이'] = (m03_test['날짜열_1년후_종료일'] - m03_test['계약날짜']).dt.days
m03_test['체크'] = m03_test['현재와시작일차이']*m03_test['현재와종료일차이']

m04['현재와시작일차이'] = (m04['계약날짜'] - m04['날짜열_13개월후']).dt.days
m04['현재와종료일차이'] = (m04['날짜열_13개월후_종료일'] - m04['계약날짜']).dt.days
m04['체크'] = m04['현재와시작일차이']*m04['현재와종료일차이']

m02[m02['체크'] >= 0]

m03[m03['체크'] >= 0]
m03_test[m03_test['체크'] >= 0]

m04[m04['체크'] >= 0]

m02.set_index('index', inplace=True)

m03.set_index('index', inplace=True)
m03_test.set_index('index', inplace=True)

m04.set_index('index', inplace=True)
# train3 = pd.concat([train2, m02['11개월전_금리'][m02['체크'] >= 0]], axis = 1)

all_data = pd.concat([all_data, m03['1년전_금리'][m03['체크'] >= 0]], axis = 1)
# test = pd.concat([all_data, m03_test['1년전_금리'][m03_test['체크'] >= 0]], axis = 1)

# train3 = pd.concat([train2, m04['13개월전_금리'][m04['체크'] >= 0]], axis = 1)


/tmp/ipykernel_469482/1385744645.py:17: FutureWarning: Passing <class 'str'> to shift is deprecated and will raise in a future version, pass Timestamp instead.
  g02['날짜열_11개월후_종료일'] = g02['날짜열_11개월후'].shift(-1, fill_value = "2027-01-01") - pd.DateOffset(days=1)
/tmp/ipykernel_469482/1385744645.py:18: FutureWarning: Passing <class 'str'> to shift is deprecated and will raise in a future version, pass Timestamp instead.
  g03['날짜열_1년후_종료일'] = g03['날짜열_1년후'].shift(-1, fill_value = "2027-01-01") - pd.DateOffset(days=1)
/tmp/ipykernel_469482/1385744645.py:19: FutureWarning: Passing <class 'str'> to shift is deprecated and will raise in a future version, pass Timestamp instead.
  g04['날짜열_13개월후_종료일'] = g04['날짜열_13개월후'].shift(-1, fill_value = "2027-01-01") - pd.DateOffset(days=1)


In [ ]:
# 결과 데이터프레임을 CSV 파일로 저장
output_filename = "../data/interest_rate.csv"
all_data.to_csv(output_filename, index=False)

print(f"Data saved to {output_filename}")

In [ ]:
all_data

### 8. 연월 변수 추가

In [ ]:
all_data['계약년'] = all_data['계약날짜'].dt.year
all_data['계약월'] = all_data['계약날짜'].dt.month

In [ ]:
# 결과 데이터프레임을 CSV 파일로 저장
output_filename = "../data/all_data.csv"
all_data.to_csv(output_filename, index=False)

print(f"Data saved to {output_filename}")